```Фамилия Имя Отчество```: ```Фархутдинов Айдар Зинфирович```

```Группа```: ```ПИ21-4```

```Билет```: ```18```


Вопрос 1. Принцип распространения значений при выполнении опреций в NumPy: общий алгоритм и примеры

In [ ]:
import numpy as np
a = np.array([1,2,3])
a*3

array([3, 6, 9])

---

Задача 2. 1. Верно ли, что средний возраст выживших людей выше, чем у не выживших пассажиров?
2. Найдите самое популярное имя среди пассажиров Титаника мужского пола

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # подключаемся к гугл диску

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/datasets/titanic.csv") # загружаем датасет в pd.DataFrame
data.head() # выводим 5 первых строк

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
agesurv = data[data['Survived']==1]['Age'].mean() # средний возраст выживших
print('Средний возраст выживших',agesurv)

Средний возраст выживших 28.343689655172415


In [ ]:
agenotsurv = data[data['Survived']==0]['Age'].mean() # средний возраст погибших
print('Средний возраст погибших',agenotsurv)

Средний возраст погибших 30.62617924528302


In [ ]:
print("Верно ли, что средний возраст выживших людей выше, чем у не выживших пассажиров?",agesurv > agenotsurv)

Верно ли, что средний возраст выживших людей выше, чем у не выживших пассажиров? False


Средний возраст выживших меньше погибших




In [ ]:
mdata = data[data['Sex']=='male']['Name'] # выбираем из датасета мужчин, выбираем столбец имя

names = mdata.str.split(',').str[1].str.split(' ').str[2] # сплитим по ",", берем 2 часть(отбрасываем Фамилию); сплитим по пробелу, берем 3 часть(отбрасываем Mr. и т.п. и отчество)
popname = names.value_counts().index[0] # подсчитываем кол-во вхождений каждого имени в датасет, берем 1 строку - это искомое имя
print('Самое популярное имя',popname)

Самое популярное имя William


Задача 3. Датасет Chinook_Sqlite.sqlite
С помощью кода на Python с использованием sqlite3 и SQL решить задачу. Реализовать функции на Python:
1. Которая возвращает все имеющиеся плейлисты.
2. Которая по имени плейлиста возвращает количество треков в нем и их суммарную продолжительность

In [ ]:
import sqlite3

In [77]:
# Функция которая возвращает все имеющиеся плейлисты
def get_all_playlist(): # возращает pd.DataFrame
  con = sqlite3.connect('/content/drive/MyDrive/datasets/Chinook_Sqlite.sqlite') #подключаемся к базе данных
  cur = con.cursor() # создание курсора
  try:
      cur.execute("""
      SELECT Name
      FROM Playlist
                  """) # выполнение SQL запроса
      fa = cur.fetchall() # получение результатов запроса
      columns = [i[0] for i in cur.description] # имена колонок
      data = pd.DataFrame(fa, columns=columns) # создание таблицы с результатом запроса
  except sqlite3.DatabaseError as error: # проверка на наличие ошибок
      print("Ошибка:", error)
  finally:
      cur.close()
      con.close() # закрываем поключения и курсор

  return data

In [78]:
get_all_playlist() # вывод всех плейлистов из БД

,Name
0,Music
1,Movies
2,TV Shows
3,Audiobooks
4,90’s Music
5,Audiobooks
6,Movies
7,Music
8,Music Videos
9,TV Shows


В выводе присутствуют плейлисты с одинаковыми именами, т.к. в базе данных не учтена уникальность имен

In [ ]:
# Функция Которая по имени плейлиста возвращает количество треков в нем и их суммарную продолжительность
def get_playlist_by_name(playlist_name): # принимает сроку playlist_name, возращает pd.DataFrame
  con = sqlite3.connect('/content/drive/MyDrive/datasets/Chinook_Sqlite.sqlite') # подключаемя к базе данных
  cur = con.cursor() # создание курсора
  try:
      cur.execute(f"""
      SELECT playlist.Name AS 'Название плейлиста', COUNT(Track.TrackId) AS 'Количество треков', SUM(Track.Milliseconds) AS 'Cуммарная продолжительность в миллисекундах'
      FROM playlist
      JOIN PlaylistTrack ON playlist.PlaylistId = PlaylistTrack.PlaylistId
      JOIN Track ON PlaylistTrack.TrackId = Track.TrackId
      WHERE playlist.Name = '{playlist_name}'
      GROUP BY playlist.Name
                  """) # выполнение SQL запроса
      fa = cur.fetchall() # получение результатов запроса
      columns = [i[0] for i in cur.description] # имена колонок
      data = pd.DataFrame(fa, columns=columns) # создание таблицы с результатом запроса
  except sqlite3.DatabaseError as error: # проверка на наличие ошибок
      print("Ошибка:", error)
  finally:
      cur.close()
      con.close() # закрываем подключения и курсор
  return data

---

In [ ]:
get_playlist_by_name('Music') # вывод кол-ва треков и их суммарной продолжительности из плейлиста Music

,Название плейлиста,Количество треков,Cуммарная продолжительность в миллисекундах
0,Music,6580,1755366166


In [79]:
get_playlist_by_name('Grunge')

,Название плейлиста,Количество треков,Cуммарная продолжительность в миллисекундах
0,Grunge,15,4122018


Решение отправить в форму: [форма для сдачи](https://forms.gle/EWdvEa8g7vrnP53n9)